In [1]:
# Installs this package locally so we can import it like any other module (see below).
!pip install -e ..
!pip install wget

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Obtaining file:///workspace/data
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Attempting uninstall: mids-plane-classification
    Found existing installation: mids-plane-classification 0.0.1
    Uninstalling mids-plane-classification-0.0.1:
      Successfully uninstalled mids-plane-classification-0.0.1
  Running setup.py develop for mids-plane-classification
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from mids_plane_classification.models.resnet18 import AircraftResNet18
from mids_plane_classification.loaders.dataloader import PlaneDataModule

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Instantiate the custom model.
model = AircraftResNet18()

# Load from the checkpoint.
checkpoint = torch.load("../mids_plane_classification/models/checkpoints/model_best.pth.tar", map_location=torch.device('cpu'))

# Load from the state dict.
model.load_state_dict(checkpoint['state_dict'])

# Set model to evaluation mode.
model.eval()

AircraftResNet18(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplac

In [4]:
dm = PlaneDataModule(train_batch_size=32, val_batch_size=32, data_dir='../data', seed=2)
dm.prepare_data()
dm.setup()
classes = dm.dataset.classes

In [5]:
import torch.backends.cudnn as cudnn
cudnn.benchmark = True

# decode the results into ([predicted class, description], probability)
def predict(input_tensor, model):
    input_batch = input_tensor.unsqueeze(0)
    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)
        sm_output = torch.nn.functional.softmax(output[0], dim=0)
    ind = torch.argmax(sm_output)
    return str(ind.item()), sm_output[ind] #([predicted class, description], probability)

def benchmark(model, input_shape=(32, 1, 256, 256), dtype='fp32', nwarmup=50, nruns=10000):
    input_data = torch.randn(input_shape)
    input_data = input_data.to("cuda")
    if dtype=='fp16':
        input_data = input_data.half()
        
    print("Warm up ...")
    with torch.no_grad():
        for _ in range(nwarmup):
            features = model(input_data)
    torch.cuda.synchronize()
    print("Start timing ...")
    timings = []
    with torch.no_grad():
        for i in range(1, nruns+1):
            start_time = time.time()
            features = model(input_data)
            torch.cuda.synchronize()
            end_time = time.time()
            timings.append(end_time - start_time)
            if i%10==0:
                print('Iteration %d/%d, avg batch time %.2f ms'%(i, nruns, np.mean(timings)*1000))

    print("Input shape:", input_data.size())
    print("Output features size:", features.size())
    print('Average throughput: %.2f images/second'%(input_shape[0]/np.mean(timings)))

In [6]:
import time
import json

# convert the model

In [9]:
import torch
import torch_tensorrt

In [10]:
trt_model_fp32 = torch_tensorrt.compile(model, inputs = [torch_tensorrt.Input((32, 3, 256, 256), dtype=torch.float32)],
    enabled_precisions = torch.float32, # Run with FP32
    workspace_size = 1 << 22
)


In [11]:

image_count = 32
for idx in range(image_count):
    print(predict(dm.val_dataset[idx][0],trt_model_fp32))

ERROR: [Torch-TensorRT] - 3: [executionContext.cpp::setBindingDimensions::947] Error Code 3: API Usage Error (Parameter check failed at: runtime/api/executionContext.cpp::setBindingDimensions::947, condition: profileMinDims.d[i] <= dimensions.d[i]. Supplied binding dimension [1,3,256,256] for bindings[0] exceed min ~ max range at index 0, maximum dimension in profile is 32, minimum dimension in profile is 32, but supplied dimension is 1.
)
ERROR: [Torch-TensorRT] - 3: [executionContext.cpp::setBindingDimensions::947] Error Code 3: API Usage Error (Parameter check failed at: runtime/api/executionContext.cpp::setBindingDimensions::947, condition: profileMinDims.d[i] <= dimensions.d[i]. Supplied binding dimension [1,3,256,256] for bindings[0] exceed min ~ max range at index 0, maximum dimension in profile is 32, minimum dimension in profile is 32, but supplied dimension is 1.
)
ERROR: [Torch-TensorRT] - 3: [executionContext.cpp::setBindingDimensions::947] Error Code 3: API Usage Error (Pa

('7', tensor(0.9999, device='cuda:0'))
('2', tensor(0.9996, device='cuda:0'))
('5', tensor(0.9993, device='cuda:0'))
('6', tensor(0.9947, device='cuda:0'))
('2', tensor(1.0000, device='cuda:0'))


ERROR: [Torch-TensorRT] - 3: [executionContext.cpp::setBindingDimensions::947] Error Code 3: API Usage Error (Parameter check failed at: runtime/api/executionContext.cpp::setBindingDimensions::947, condition: profileMinDims.d[i] <= dimensions.d[i]. Supplied binding dimension [1,3,256,256] for bindings[0] exceed min ~ max range at index 0, maximum dimension in profile is 32, minimum dimension in profile is 32, but supplied dimension is 1.
)
ERROR: [Torch-TensorRT] - 3: [executionContext.cpp::setBindingDimensions::947] Error Code 3: API Usage Error (Parameter check failed at: runtime/api/executionContext.cpp::setBindingDimensions::947, condition: profileMinDims.d[i] <= dimensions.d[i]. Supplied binding dimension [1,3,256,256] for bindings[0] exceed min ~ max range at index 0, maximum dimension in profile is 32, minimum dimension in profile is 32, but supplied dimension is 1.
)
ERROR: [Torch-TensorRT] - 3: [executionContext.cpp::setBindingDimensions::947] Error Code 3: API Usage Error (Pa

('3', tensor(0.9981, device='cuda:0'))
('0', tensor(1.0000, device='cuda:0'))
('3', tensor(0.9483, device='cuda:0'))
('0', tensor(0.9947, device='cuda:0'))
('8', tensor(0.9944, device='cuda:0'))
('2', tensor(0.9842, device='cuda:0'))
('4', tensor(0.9995, device='cuda:0'))
('1', tensor(0.9999, device='cuda:0'))


ERROR: [Torch-TensorRT] - 3: [executionContext.cpp::setBindingDimensions::947] Error Code 3: API Usage Error (Parameter check failed at: runtime/api/executionContext.cpp::setBindingDimensions::947, condition: profileMinDims.d[i] <= dimensions.d[i]. Supplied binding dimension [1,3,256,256] for bindings[0] exceed min ~ max range at index 0, maximum dimension in profile is 32, minimum dimension in profile is 32, but supplied dimension is 1.
)
ERROR: [Torch-TensorRT] - 3: [executionContext.cpp::setBindingDimensions::947] Error Code 3: API Usage Error (Parameter check failed at: runtime/api/executionContext.cpp::setBindingDimensions::947, condition: profileMinDims.d[i] <= dimensions.d[i]. Supplied binding dimension [1,3,256,256] for bindings[0] exceed min ~ max range at index 0, maximum dimension in profile is 32, minimum dimension in profile is 32, but supplied dimension is 1.
)
ERROR: [Torch-TensorRT] - 3: [executionContext.cpp::setBindingDimensions::947] Error Code 3: API Usage Error (Pa

('6', tensor(1.0000, device='cuda:0'))
('3', tensor(0.9985, device='cuda:0'))
('2', tensor(1.0000, device='cuda:0'))
('9', tensor(0.9999, device='cuda:0'))
('3', tensor(0.5646, device='cuda:0'))
('0', tensor(0.9999, device='cuda:0'))
('4', tensor(0.3823, device='cuda:0'))
('4', tensor(0.9642, device='cuda:0'))


ERROR: [Torch-TensorRT] - 3: [executionContext.cpp::setBindingDimensions::947] Error Code 3: API Usage Error (Parameter check failed at: runtime/api/executionContext.cpp::setBindingDimensions::947, condition: profileMinDims.d[i] <= dimensions.d[i]. Supplied binding dimension [1,3,256,256] for bindings[0] exceed min ~ max range at index 0, maximum dimension in profile is 32, minimum dimension in profile is 32, but supplied dimension is 1.
)
ERROR: [Torch-TensorRT] - 3: [executionContext.cpp::setBindingDimensions::947] Error Code 3: API Usage Error (Parameter check failed at: runtime/api/executionContext.cpp::setBindingDimensions::947, condition: profileMinDims.d[i] <= dimensions.d[i]. Supplied binding dimension [1,3,256,256] for bindings[0] exceed min ~ max range at index 0, maximum dimension in profile is 32, minimum dimension in profile is 32, but supplied dimension is 1.
)
ERROR: [Torch-TensorRT] - 3: [executionContext.cpp::setBindingDimensions::947] Error Code 3: API Usage Error (Pa

('4', tensor(0.9999, device='cuda:0'))
('0', tensor(1.0000, device='cuda:0'))
('4', tensor(0.9995, device='cuda:0'))
('6', tensor(0.9980, device='cuda:0'))
('5', tensor(0.9999, device='cuda:0'))
('8', tensor(0.9998, device='cuda:0'))
('10', tensor(0.3781, device='cuda:0'))
('5', tensor(0.9972, device='cuda:0'))
('0', tensor(1.0000, device='cuda:0'))
('10', tensor(0.6095, device='cuda:0'))
('2', tensor(0.9918, device='cuda:0'))


ERROR: [Torch-TensorRT] - 3: [executionContext.cpp::setBindingDimensions::947] Error Code 3: API Usage Error (Parameter check failed at: runtime/api/executionContext.cpp::setBindingDimensions::947, condition: profileMinDims.d[i] <= dimensions.d[i]. Supplied binding dimension [1,3,256,256] for bindings[0] exceed min ~ max range at index 0, maximum dimension in profile is 32, minimum dimension in profile is 32, but supplied dimension is 1.
)
ERROR: [Torch-TensorRT] - 3: [executionContext.cpp::setBindingDimensions::947] Error Code 3: API Usage Error (Parameter check failed at: runtime/api/executionContext.cpp::setBindingDimensions::947, condition: profileMinDims.d[i] <= dimensions.d[i]. Supplied binding dimension [1,3,256,256] for bindings[0] exceed min ~ max range at index 0, maximum dimension in profile is 32, minimum dimension in profile is 32, but supplied dimension is 1.
)
ERROR: [Torch-TensorRT] - 3: [executionContext.cpp::setBindingDimensions::947] Error Code 3: API Usage Error (Pa

In [12]:
benchmark(trt_model_fp32, input_shape=(32, 3, 256, 256), nruns=100)

Warm up ...
Start timing ...
Iteration 10/100, avg batch time 27.29 ms
Iteration 20/100, avg batch time 27.39 ms
Iteration 30/100, avg batch time 27.41 ms
Iteration 40/100, avg batch time 27.49 ms
Iteration 50/100, avg batch time 27.51 ms
Iteration 60/100, avg batch time 27.47 ms
Iteration 70/100, avg batch time 27.47 ms
Iteration 80/100, avg batch time 27.44 ms
Iteration 90/100, avg batch time 27.45 ms
Iteration 100/100, avg batch time 27.45 ms
Input shape: torch.Size([32, 3, 256, 256])
Output features size: torch.Size([32, 11])
Average throughput: 1165.63 images/second


In [14]:
model.eval().cuda()
benchmark(model, input_shape=(32, 3, 256, 256), nruns=100)

Warm up ...
Start timing ...
Iteration 10/100, avg batch time 30.39 ms
Iteration 20/100, avg batch time 30.42 ms
Iteration 30/100, avg batch time 30.35 ms
Iteration 40/100, avg batch time 30.32 ms
Iteration 50/100, avg batch time 30.39 ms
Iteration 60/100, avg batch time 30.39 ms
Iteration 70/100, avg batch time 30.36 ms
Iteration 80/100, avg batch time 30.37 ms
Iteration 90/100, avg batch time 30.38 ms
Iteration 100/100, avg batch time 30.37 ms
Input shape: torch.Size([32, 3, 256, 256])
Output features size: torch.Size([32, 11])
Average throughput: 1053.57 images/second
